In [ ]:
import tensorflow as tf

In [ ]:
%run training.ipynb

In [ ]:
# Implementation of the Carlini & Wagner Attack on the model trained in training.ipynb

def cw_attack(model, img, label, eps, lr, max_iter):
    
    # Initialization
    w = tf.Variable(tf.zeros_like(img), trainable=True)
    optimizer = tf.optimizers.Adam(learning_rate=lr)
    
    new_img = 0.5 * (tf.tanh(w) + 1)
    output = model(new_img)
    real = tf.reduce_max(tf.multiply(output, label))
    other = tf.reduce_max(tf.multiply(output, (1 - label)))
    loss1 = tf.maximum(0.0, other - real)
    loss2 = tf.norm(tf.reshape(new_img - img, [-1]))
    loss = eps * loss1 + loss2

    # Optimization
    for _ in range(max_iter):
        with tf.GradientTape() as t:
        gradient = t.gradient(loss, [w])
        optimizer.apply_gradients(zip(gradient, [w]))

    adv_img = 0.5 * (tf.tanh(w) + 1)
    return adv_img

In [ ]:
#we separate labels and images from our test set
test_images = [] 
test_labels = [] 
for i in range(len(test_set)):
    test_images.append(test_set[i][0])
    test_labels.append(int(test_set[i][1][-2:]))
    
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
# test on one image
eps = 0.01
lr = 0.01
max_iter = 1000
img_adversarial = np.array(cw_attack(model, test_images[4:5], tf.one_hot(test_labels[4], 16), eps, lr, max_iter))
img_adversarial = img_adversarial[0, :,:,:]

tf.keras.utils.array_to_img(test_images[4])
tf.keras.utils.array_to_img(img_adversarial)

In [ ]:
# test on the whole test_set (accuracy test)
eps = 0.01
lr = 0.01
max_iter = 100

#use this code to train on the whole test_set but it is a bit long
adv_img = np.zeros_like(test_images)
for i in range(len(test_images)):
    adv_img[i] = cw_attack(model, test_images[i:i+1], tf.one_hot(test_labels[i], 16), eps, lr, max_iter)

# Evaluate the model on adversarial examples
test_loss, test_acc = model.evaluate(adv_img, test_labels, verbose=2)
print('\nTest accuracy on adversarial images:', test_acc)